**Your work will be evaluated according to the following criteria:**
- Project Structure and Notebook(s) Quality (4/20)
- Data Exploration & Initial Preprocessing (4/20)
- Regression Benchmarking and Optimization (7/20)
- Open-Ended Section (4/20)
- Deployment (1/20)
- Extra Point: Have Project Be Publicly Available on GitHub (1/20)


In [0]:
# TODO Brainstorm + Implementation of Ideas for open ended Section (Several can get explored):
#   1. Create a classification Model, that predicts if a dataset is gonna be a price outlier (outlier flag) (SAMUEL)
#   2. Jan:
#   3. Elias: 
#   4. Lukas: 

<div style="
    background: rgba(25, 25, 25, 0.55);
    backdrop-filter: blur(16px) saturate(150%);
    -webkit-backdrop-filter: blur(16px) saturate(150%);
    border: 1px solid rgba(255, 255, 255, 0.12);
    border-radius: 18px;
    padding: 45px 30px;
    text-align: center;
    font-family: 'Inter', 'Segoe UI', 'Helvetica Neue', Arial, sans-serif;
    color: #e0e0e0;
    box-shadow: 0 0 30px rgba(0, 0, 0, 0.35);
    margin: 40px auto;
    max-width: 800px;
">

  <h1 style="
      font-size: 2.8em;
      font-weight: 700;
      margin: 0 0 8px 0;
      letter-spacing: -0.02em;
      background: linear-gradient(90deg, #00e0ff, #9c7eff);
      -webkit-background-clip: text;
      -webkit-text-fill-color: transparent;
  ">
      Machine Learning Project
  </h1>

  <h2 style="
      font-size: 1.6em;
      font-weight: 500;
      margin: 0 0 25px 0;
      color: #b0b0b0;
      letter-spacing: 0.5px;
  ">
      Cars 4 You - Predicting Car Prices
  </h2>

  <p style="
      font-size: 1.25em;
      font-weight: 500;
      color: #c0c0c0;
      margin-bottom: 6px;
  ">
      Group 5 - Lukas Belser, Samuel Braun, Elias Karle, Jan Thier
  </p>

  <p style="
      font-size: 1.05em;
      font-weight: 400;
      color: #8a8a8a;
      font-style: italic;
      letter-spacing: 0.5px;
  ">
      Machine Learning End Results · 22.12.2025
  </p>
</div>


# **Table of Contents**

TODO finish toc > at the end of project

Process to test new features: (TODO remove before submission)

1. Add new Features in cell Feature Engineering
2. Go into Preprocessing and add the features in either num, log or cat
3. Run the RandomSearch // Hyperparameter Tuning for at least HGB and RF and see if MAE gets improved compared to previous results
4. If MAE gets not improved, comment in cell below Feature Engineering that you tested those features (+results?) and on which Models - and remove everything
5. If MAE gets improved, find out via Feature Importance (Shap Values) which Feature was responsible + document it, remove the other features that have negative impact
6. comment below the Hyperparameter Tuning cell of the model the new achieved results + all the features you used for that + the hyperparameters
7. save the results in a new model with joblib, name it correctly
8. push to kaggle
9. push to GIT + document everything

### 1. Import Packages and Data

#### 1.1 Import Required Packages

In [0]:
# TODO remove pip installs before submission

In [0]:
pip install kaggle

In [0]:
pip install shap

In [0]:
pip install -U scikit-learn

In [0]:
# Import and load Data # TODO remove unused imports

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import joblib
import seaborn as sns
import shap


from sklearn.feature_selection import VarianceThreshold, RFE, chi2, RFECV
from scipy.stats import spearmanr, uniform, loguniform, randint
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, FunctionTransformer, RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin


from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor, StackingRegressor
from sklearn.svm import SVR

from car_functions import clean_car_dataframe, GroupMedianImputer, cv_target_encode, add_price_anchor_features, print_metrics


#### 1.2 Load Datasets

In [0]:
df_cars_train = pd.read_csv("train.csv")
df_cars_test = pd.read_csv("test.csv")

#### 1.3 Kaggle Setup

In [0]:
# Kaggle API Connect => everyone has to do this himself, with his own kaggle.json api token

# Folder containing kaggle.json
os.environ['KAGGLE_CONFIG_DIR'] = "/Workspace/Users/20250355@novaims.unl.pt"

# Test
!echo $KAGGLE_CONFIG_DIR

### 2. Data Cleaning, Feature Engineering, Split & Preprocessing

#### 2.1 Data Cleaning

In [0]:
# Outlier Preprocessing, Missing Value Handling and Decision justifying happens here
df_cars_train = clean_car_dataframe(df_cars_train)
df_cars_test = clean_car_dataframe(df_cars_test)


# Safety Check: print unique values of all columns of df_cars_train // df_cars_test to see if data cleaning worked and if there are still odd values
for col in df_cars_train.columns:
    print(col, df_cars_train[col].unique())
print("X"*150); print("-"*150); print("X"*150)
for col in df_cars_test.columns:
    print(col, df_cars_test[col].unique())

#### 2.2 Feature Engineering (a)

In [0]:
# 1. Base Feature Creation

# Car Age: Newer cars usually have higher prices, models prefer linear features
df_cars_train['age'] = 2020 - df_cars_train['year']
df_cars_test['age']  = 2020 - df_cars_test['year']

# Miles per Year: Normalizes mileage by age (solves multicollinearity between year and mileage)
df_cars_train['miles_per_year'] = df_cars_train['mileage'] / df_cars_train['age'].replace({0: np.nan})
df_cars_train['miles_per_year'] = df_cars_train['miles_per_year'].fillna(df_cars_train['mileage'])

df_cars_test['miles_per_year'] = df_cars_test['mileage'] / df_cars_test['age'].replace({0: np.nan})
df_cars_test['miles_per_year'] = df_cars_test['miles_per_year'].fillna(df_cars_test['mileage'])

# Interaction Terms: Capture non-linear effects between engine and other numeric features
df_cars_train['age_x_engine'] = df_cars_train['age'] * df_cars_train['engineSize'].fillna(0)
df_cars_test['age_x_engine']  = df_cars_test['age']  * df_cars_test['engineSize'].fillna(0)

df_cars_train['mpg_x_engine'] = df_cars_train['mpg'].fillna(0) * df_cars_train['engineSize'].fillna(0)
df_cars_test['mpg_x_engine']  = df_cars_test['mpg'].fillna(0)  * df_cars_test['engineSize'].fillna(0)

# tax per engine
df_cars_train['tax_per_engine'] = df_cars_train['tax'] / df_cars_train['engineSize'].replace({0: np.nan})
df_cars_train['tax_per_engine'] = df_cars_train['tax_per_engine'].fillna(df_cars_train['tax'])

df_cars_test['tax_per_engine'] = df_cars_test['tax'] / df_cars_test['engineSize'].replace({0: np.nan})
df_cars_test['tax_per_engine'] = df_cars_test['tax_per_engine'].fillna(df_cars_test['tax'])

# MPG per engineSize to represent the efficiency
df_cars_train['mpg_per_engine'] = df_cars_train['mpg'] / df_cars_train['engineSize'].replace({0: np.nan})
df_cars_train['mpg_per_engine'] = df_cars_train['mpg_per_engine'].fillna(df_cars_train['mpg'])

df_cars_test['mpg_per_engine'] = df_cars_test['mpg'] / df_cars_test['engineSize'].replace({0: np.nan})
df_cars_test['mpg_per_engine'] = df_cars_test['mpg_per_engine'].fillna(df_cars_test['mpg'])


# 2. Model Frequency: Popular models tend to have stable demand and prices
model_freq = df_cars_train['model'].value_counts(normalize=True).to_dict()

df_cars_train['model_freq'] = df_cars_train['model'].map(model_freq).fillna(0.0) # TODO chat added fillna
df_cars_test['model_freq']  = df_cars_test['model'].map(model_freq).fillna(0.0)


# 3. Relative Age (within brand): newer/older than brand median year
brand_median_age = df_cars_train.groupby('Brand')['age'].median().to_dict()

df_cars_train['age_rel_brand'] = df_cars_train['age'] - df_cars_train['Brand'].map(brand_median_age)
df_cars_test['age_rel_brand']  = df_cars_test['age']  - df_cars_test['Brand'].map(brand_median_age)


# 4. CV-Safe Target Encodings (no leakage): category means
for col, m in [('model', 100), ('Brand', 30), ('fuelType', 20), ('transmission', 20)]:
    tr_enc, te_enc = cv_target_encode(df_cars_train, df_cars_test, col, ycol='price', m=m)
    df_cars_train[f'{col}_te'] = tr_enc
    df_cars_test[f'{col}_te']  = te_enc

#### 2.3 Data Split

In [0]:
# Split Data, Stratify not necessary due to regression problem and Cross Validation later
X = df_cars_train.drop(columns='price')
y = df_cars_train['price']

X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.2, 
                                                  random_state = 42, 
                                                  shuffle = True)

#### 2.2 Feature Engineering (b)

In [0]:
# 5. Brand/Model/Fuel/Transmission price anchors without leaking validation targets

# We now compute price anchors using only the training fold (X_train + y_train) 
# and then map them onto X_val. This avoids the subtle leakage where validation
# rows help define their own anchor via the global group median.

# Rebuild small train/val dataframes with the target so the helper can see `price`
train_tmp = X_train.copy()
train_tmp["price"] = y_train

val_tmp = X_val.copy()
val_tmp["price"] = y_val

# Compute anchors based only on training labels and map them to validation
train_tmp, val_tmp = add_price_anchor_features(
    train_tmp,
    val_tmp,
    target_col="price"
)

# Drop target again: X_train/X_val now include brand_*_price + *_anchor columns
X_train = train_tmp.drop(columns=["price"])
X_val   = val_tmp.drop(columns=["price"])


#### 2.4 Preprocessing

In [0]:
def to_float_array(x):
    """Convert input to float array."""
    return np.array(x, dtype=float)

In [0]:
# PIPELINE WITH preprocessor_orig CONTAINING ONLY ORIGINAL FEATURES


orig_numeric_features = [
    "year", "mileage", "tax", "mpg", "engineSize", "paintQuality", "previousOwners"
]
orig_categorical_features = ["Brand", "model", "transmission", "fuelType"]

numeric_transformer_orig = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),    # simple global median imputation
    ("to_float", FunctionTransformer(to_float_array)),
    ("scaler", StandardScaler())
])

categorical_transformer_orig = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),  # fill by mode instead of Unknown
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))  # One-hot encoding
])

preprocessor_orig = ColumnTransformer([
    ("num", numeric_transformer_orig, orig_numeric_features),
    ("cat", categorical_transformer_orig, orig_categorical_features)
])

preprocessor_orig.fit(X_train)

In [0]:
# PIPELINE WITH preprocessor_fe CONTAINING ENGINEERED FEATURES

# Custom Written GroupMedianImputer to get Brand and Model specific Medians
# Uses target-encoded Brand_te/model_te as grouping cols (present in numeric_features)
group_imputer = GroupMedianImputer(group_cols=["Brand_te", "model_te"])

numeric_features = [
    "age", "age_rel_brand", "tax", "mpg", "engineSize", "paintQuality", "previousOwners", "model_freq",
    "brand_med_price_anchor", "model_med_price_anchor", "brand_fuel_med_price_anchor", "brand_trans_med_price_anchor",
    "age_x_engine", "mpg_x_engine",
    "model_te", "Brand_te", "fuelType_te", "transmission_te",
    "tax_per_engine", "mpg_per_engine"
]
log_features = ["mileage", "miles_per_year"]  # TODO other num columns here better?!
categorical_features = ["Brand", "model", "transmission", "fuelType"]

# left out columns: year (age is better), hasDamage (unsure what the two values 0 and NaN mean)


log_transformer_fe = Pipeline([
    # Hierarchical imputation on Brand_te/model_te, then log-transform
    ("group_impute", GroupMedianImputer(group_cols=["Brand_te", "model_te"])),
    ("to_float", FunctionTransformer(to_float_array)),
    ("log", FunctionTransformer(np.log1p, validate=False)),  # log1p handles zeros safely
    ("scaler", StandardScaler())
])

numeric_transformer_fe = Pipeline([
    ("group_impute", GroupMedianImputer(group_cols=["Brand_te", "model_te"])),
    ("to_float", FunctionTransformer(to_float_array)),
    ("scaler", StandardScaler())
])

categorical_transformer_fe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),  # fill by mode instead of Unknown
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))  # One-hot encoding
])

# ColumnTransformer that uses all engineered features
preprocessor_fe = ColumnTransformer([
    ("log", log_transformer_fe, log_features),
    ("num", numeric_transformer_fe, numeric_features),
    ("cat", categorical_transformer_fe, categorical_features)
])

preprocessor_fe.fit(X_train)

# EXPLANATIONS:
# 1) Pipeline bundles preprocessing + model training:
#       > Ensures all preprocessing happens inside cross-validation folds (no data leakage)
#       > Keeps the entire workflow reproducible — scaling, encoding, and modeling are learned together
#       > After .fit(), the final model automatically knows how to preprocess new unseen data
#       > When saving with joblib, the entire preprocessing (imputers, scalers, encoders) and model are stored together

# 2) The ColumnTransformer applies different transformations to subsets of features:
#       > Numeric Features arehandled by our custom GroupMedianImputer (domain-aware filling)
#           - Missing numeric values are imputed hierarchically:
#           1. By (Brand, model)
#           2. If missing model by Brand
#           3. If missing Brand by global median
#       > This approach captures brand/model-level patterns (e.g. BMWs have similar engine sizes)
#       > After imputation, StandardScaler standardizes all numeric features
#
#       > Log Features use the same group-median imputation, followed by log1p() transformation
#           - log1p() compresses large, skewed values (like mileage or price-related features), stabilizing variance and helping linear models perform better
#           - StandardScaler then scales them to zero mean and unit variance
#
#       > Categorical Features are handled by SimpleImputer + OneHotEncoder
#           - SimpleImputer fills missing categorical values with the most frequent (mode) value.
#             (Alternative would be “Unknown”, but mode keeps categories realistic, e.g. most cars in a model share the same transmission)
#           - OneHotEncoder converts each categorical label (Brand, model, etc.) into binary dummy variables
#             This lets the model use category information numerically without implying order
#
# 3) Overall:
#       > The pipeline ensures consistent preprocessing across training, validation, and test data.
#       > It combines domain knowledge (brand/model-aware imputation) with robust numerical scaling.
#       > Linear models (ElasticNet, Ridge, Lasso) and tree models (HistGradientBoosting, RandomForest)
#           can now learn from the same standardized, clean, and information-rich feature space.

### 3. Feature Selection

In [0]:
X_train_proc = preprocessor_fe.transform(X_train)

feature_names_all = []
for name, trans, cols in preprocessor_fe.transformers_:
    if name != 'remainder':
        if hasattr(trans, 'get_feature_names_out'):
            # for categorical OHE
            try:
                feature_names_all.extend(trans.get_feature_names_out(cols))
            except:
                feature_names_all.extend(cols)
        else:
            feature_names_all.extend(cols)

X_df = pd.DataFrame(X_train_proc, columns=feature_names_all)


# Variance Threshold
vt = VarianceThreshold(threshold=0.0)
vt.fit(X_df)
vt_deselect = [f for f, keep in zip(feature_names_all, vt.get_support()) if not keep]
print("Features to deselect according to VarianceThreshold:", vt_deselect)


# Spearman correlation (numeric + log only)
numeric_log = numeric_features + log_features
spearman_deselect = []
for f in numeric_log:
    if f in X_df.columns:
        corr, _ = spearmanr(X_df[f], y_train)
        if abs(corr) <= 0.05:
            spearman_deselect.append(f)
print("Features to deselect according to Spearman correlation:", spearman_deselect)


# Chi2 (categorical only, must be non-negative)
cat_cols = [c for c in X_df.columns if c not in numeric_log]
X_cat = X_df[cat_cols].astype(float)
chi2_vals, _ = chi2(X_cat, y_train)
chi2_deselect = [f for f, val in zip(cat_cols, chi2_vals) if val <= 0]
print("Features to deselect according to Chi²:", chi2_deselect)


In [0]:
# Numeric/log features for linear models
linear_numeric_features = [f for f in numeric_features + log_features if f not in spearman_deselect]

preprocessor_linear = ColumnTransformer([
    ("num", numeric_transformer_fe, linear_numeric_features),
    ("cat", categorical_transformer_fe, categorical_features)
], remainder="drop")

# => use preprocessor_linear for linear model setup; since tree models are indifferent to irrelevant features

### 4. Model Evaluation Metrics, Baselining, Setup

In [0]:
# Model evaluation metrics used throughout this analysis:
#   MAE: Mean Absolute Error: average absolute deviation between predicted and true car prices
#           Easy to interpret in pounds, same metric used by Kaggle competition
#   RMSE: Root Mean Squared Error: sensitive to outliers, helps identify large prediction errors  
#   R²: Coefficient of determination: proportion of variance explained by the model
#           1.0 = perfect predictions, 0.0 = same as predicting mean, < 0.0 = worse than mean
#
# These metrics are appropriate for regression problems predicting continuous variables (car prices)
# We define them in the method file car_functions.py

In [0]:
# Absolute basic baselining with the mean and median

mean_pred = y_train.mean()
median_pred = y_train.median()

print("baseline mean predictor: ")
print_metrics(y_val, [mean_pred]*len(y_val))
# MAE: 6976.3626 | RMSE: 92839550.2849 | R2: -0.0000

print("-"*150)

print("baseline median predictor: ") 
print_metrics(y_val, [median_pred]*len(y_val))
# MAE: 6751.1604 | RMSE: 97557866.6363 | R2: -0.0508

In [0]:
### LINEAR MODEL (ElasticNet)

# Original-features pipeline
elastic_pipe_orig = Pipeline([
    ("preprocess", preprocessor_orig),
    ("model", ElasticNet(
        alpha=0.01,
        l1_ratio=0.5,
        max_iter=30000,
        tol=1e-4,
        selection="cyclic",
        random_state=42
    ))
])

# Engineered-features pipeline
elastic_pipe_fe = Pipeline([
    ("preprocess", preprocessor_linear), 
    ("model", ElasticNet(
        alpha=0.01,
        l1_ratio=0.5,
        max_iter=30000,
        tol=1e-4,
        selection="cyclic",
        random_state=42
    ))
])


### TREE MODELS

# HistGradientBoostingRegressor

hgb_pipe_orig = Pipeline([
    ("preprocess", preprocessor_orig),
    ("model", HistGradientBoostingRegressor(
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=20,
        l2_regularization=0.5,
        random_state=42
    ))
])

hgb_pipe_fe = Pipeline([
    ("preprocess", preprocessor_fe),
    ("model", HistGradientBoostingRegressor(
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=20,
        l2_regularization=0.5,
        random_state=42
    ))
])


# RandomForestRegressor

rf_pipe_orig = Pipeline([
    ("preprocess", preprocessor_orig),
    ("model", RandomForestRegressor(
        n_estimators=300,
        max_depth=None,
        min_samples_split=3,
        min_samples_leaf=2,
        max_features="sqrt",
        bootstrap=True,
        n_jobs=1,
        random_state=42
    ))
])

rf_pipe_fe = Pipeline([
    ("preprocess", preprocessor_fe),
    ("model", RandomForestRegressor(
        n_estimators=300,
        max_depth=None,
        min_samples_split=3,
        min_samples_leaf=2,
        max_features="sqrt",
        bootstrap=True,
        n_jobs=1,
        random_state=42
    ))
])


# ExtraTreesRegressor

et_pipe_orig = Pipeline([
    ("preprocess", preprocessor_orig),
    ("model", ExtraTreesRegressor(
        n_estimators=400,
        max_depth=None,
        min_samples_leaf=2,
        max_features="sqrt",
        bootstrap=False,
        n_jobs=1,
        random_state=42
    ))
])

et_pipe_fe = Pipeline([
    ("preprocess", preprocessor_fe),
    ("model", ExtraTreesRegressor(
        n_estimators=400,
        max_depth=None,
        min_samples_leaf=2,
        max_features="sqrt",
        bootstrap=False,
        n_jobs=1,
        random_state=42
    ))
])


### KERNEL-BASED MODEL (SVR)

svr_pipe_orig = Pipeline([
    ("preprocess", preprocessor_orig),
    ("model", SVR(
        kernel="rbf",
        C=10,
        epsilon=0.1,
        gamma="scale"
    ))
])

svr_pipe_fe = Pipeline([
    ("preprocess", preprocessor_fe),
    ("model", SVR(
        kernel="rbf",
        C=10,
        epsilon=0.1,
        gamma="scale"
    ))
])


# ENSEMBLE META MODEL (Stacking)

stack_pipe_orig = StackingRegressor(
    estimators=[
        ("elastic_orig", elastic_pipe_orig),
        ("hgb_orig", hgb_pipe_orig),
        ("rf_orig", rf_pipe_orig),
    ],
    final_estimator=HistGradientBoostingRegressor(
        learning_rate=0.05,
        max_depth=5,
        l2_regularization=0.5,
        random_state=42
    ),
    passthrough=False,   # <- was True: disable raw-X passthrough to avoid string->float error
    n_jobs=1
)

stack_pipe_fe = StackingRegressor(
    estimators=[
        ("elastic_fe", elastic_pipe_fe),
        ("hgb_fe", hgb_pipe_fe),
        ("rf_fe", rf_pipe_fe),
    ],
    final_estimator=HistGradientBoostingRegressor(
        learning_rate=0.05,
        max_depth=5,
        l2_regularization=0.5,
        random_state=42
    ),
    passthrough=False,   # <- same here
    n_jobs=1
)

### 5. Model Evaluation and Hyperparameter Tuning

In [0]:
# First evaluation of metrics based on original and engineered feature pipeline to decide how to proceed

models_orig = {
    "ElasticNet_orig": elastic_pipe_orig,
    "HGB_orig": hgb_pipe_orig,
    "RF_orig": rf_pipe_orig,
    "ET_orig": et_pipe_orig,
    "SVR_orig": svr_pipe_orig,
    "Stack_orig": stack_pipe_orig,
}

models_fe = {
    "ElasticNet_fe": elastic_pipe_fe,
    "HGB_fe": hgb_pipe_fe,
    "RF_fe": rf_pipe_fe,
    "ET_fe": et_pipe_fe,
    "SVR_fe": svr_pipe_fe,
    "Stack_fe": stack_pipe_fe,
}

results = []

for name, model in {**models_orig, **models_fe}.items():
    print(f"Fitting {name} ...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    mae = mean_absolute_error(y_val, y_pred)
    rmse = mean_squared_error(y_val, y_pred)  # keep consistent with print_metrics
    r2 = r2_score(y_val, y_pred)

    results.append({
        "model": name,
        "feature_set": "original" if name.endswith("_orig") else "engineered",
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2,
    })

results_df = (
    pd.DataFrame(results)
      .sort_values(["feature_set", "MAE"])
      .reset_index(drop=True)
)

print(results_df)


After a first run comparing the original feature pipeline and the engineered feature pipeline for all models, we decided to focus on RandomForest and HistGradientBoost. 

They seem to have the best prediction performance for now. StackingRegressor currently performs best, but since it is blending existing models, we will focus on that and reevaluatedin the end.

With ExtraTrees and SVR really underperforming, we decide not to do Hyperparameter Tuning.

##### 5.1 ElasticNet

In [0]:
# Hyperparameter Tuning: ElasticNet

elastic_param_grid = {
    "model__alpha": [0.001],    # also tried 0.01, 0.05, 0.1, 0.5
    "model__l1_ratio": [0.9]    # also tried 0.1, 0.3, 0.5, 0.7  
}

elastic_grid = GridSearchCV(
    elastic_pipe_fe, 
    param_grid=elastic_param_grid,
    cv=5,
    scoring="neg_mean_absolute_error",
    n_jobs=1,
    verbose=1
)

elastic_grid.fit(X_train, y_train)
elastic_best = elastic_grid.best_estimator_
elastic_val_pred = elastic_best.predict(X_val)


print("ElasticNet Results: ")
print_metrics(y_val, elastic_val_pred)
print("Best ElasticNet params:", elastic_grid.best_params_)

# ElasticNet Results: 
# MAE: 2353.9112 | RMSE: 13356867.7860 | R2: 0.8534
# Best ElasticNet params: {'model__alpha': 0.001, 'model__l1_ratio': 0.9}

In [0]:
# Wrapper Feature Selection (RFECV) for ElasticNet — uses preprocessor_linear and existing print_metrics

elastic_base = ElasticNet(
    alpha=0.001,     # from your best params
    l1_ratio=0.9,    # from your best params
    max_iter=30000,
    tol=1e-4,
    selection="cyclic",
    random_state=42
)

rfecv_pipe_linear = Pipeline([
    ("preprocess", preprocessor_linear),  # uses linear_numeric_features + categorical_features
    ("rfecv", RFECV(
        estimator=elastic_base,
        step=10,                               # remove 10 features per iteration
        cv=3,                                 # consistent with your tuning
        scoring="neg_mean_absolute_error",    # same metric
        n_jobs=1,
        min_features_to_select=10             # adjust as needed
    )),
    ("model", ElasticNet(                    # final model with same params
        alpha=0.001,
        l1_ratio=0.9,
        max_iter=30000,
        tol=1e-4,
        selection="cyclic",
        random_state=42
    ))
])

rfecv_pipe_linear.fit(X_train, y_train)
val_pred_linear_rfecv = rfecv_pipe_linear.predict(X_val)
print("ElasticNet with RFECV (Wrapper Selection):")
print_metrics(y_val, val_pred_linear_rfecv)a

In [0]:
# Show selected (post-preprocessing) feature names
pre = rfecv_pipe_linear.named_steps["preprocess"]
support_mask = rfecv_pipe_linear.named_steps["rfecv"].support_

feature_names_linear = (
    linear_numeric_features
    + list(pre.named_transformers_["cat"]
              .named_steps["encoder"]
              .get_feature_names_out(categorical_features))
)

selected_features_linear = [n for n, keep in zip(feature_names_linear, support_mask) if keep]
print(f"Selected {len(selected_features_linear)} features:")
for n in selected_features_linear:
    print("-", n)

RFECV selected 143/143 features, meaning the best cross‑validated MAE was achieved with the full post‑preprocessing feature set and dropping any feature hurt performance. 

##### 5.2 HistGradientBoost

In [0]:
# Define a function to use it here and potentially use it later for a final hyperparameter tuning after feature selection again
def hgb_hyperparameter_tuning(hgb_estimator, n_iter):
    # hgb_param_dist_old = {
    #     "model__learning_rate": uniform(0.01, 0.09),       # samples values
    #     "model__max_leaf_nodes": randint(20, 120),         # tries between 20–120 leaves
    #     "model__min_samples_leaf": randint(2, 20),         # samples leaf sizes between 2–20
    #     "model__max_iter": randint(400, 1000),             # tries 400–1000 iterations
    #     "model__l2_regularization": uniform(0.0, 1.0)      # samples small regularization values
    # }

    # optimized the parameter distributions based on previous runs to focus search space
    hgb_param_dist = {
        "model__learning_rate": [0.06389789198396824],
        "model__max_leaf_nodes": [105],
        "model__min_samples_leaf": [3],
        "model__max_iter": [642],
        "model__l2_regularization": [0.942853570557981],
        "model__early_stopping": [True],
        "model__validation_fraction": [0.1],
        "model__n_iter_no_change": [20],
        "model__random_state":[42]
    }
    

    cv = KFold(n_splits=3, shuffle=True, random_state=42)  # 3-fold for faster runtime

    # Randomized search setup
    hgb_random = RandomizedSearchCV(
        estimator=hgb_estimator,
        param_distributions=hgb_param_dist,
        n_iter=n_iter,                         # number of random combinations to try
        scoring="neg_mean_absolute_error",     # optimize for MAE (primary metric)
        cv=cv,
        n_jobs=1,
        random_state=42,
        verbose=3
    )

    # Fit the search
    hgb_random.fit(X_train, y_train)
    return hgb_random

In [0]:
# Don't run this cell unless you want to do hyperparameter tuning again (very time consuming)
hgb_random = hgb_hyperparameter_tuning(hgb_pipe_fe, n_iter=1) 
# Get best model
hgb_best = hgb_random.best_estimator_
print("Best Params:", hgb_random.best_params_)

# Evaluate on validation set
hgb_val_pred = hgb_random.best_estimator_.predict(X_val)
print_metrics(y_val, hgb_val_pred)

##### 5.3 RandomForest

In [0]:
# Hyperparameter Tuning: RandomForest

# Old parameter distribution
# rf_param_dist_old = {
#     "model__n_estimators": randint(200, 600),        # number of trees
#     "model__max_depth": randint(5, 40),              # depth of each tree
#     "model__min_samples_split": randint(2, 10),      # min samples to split an internal node
#     "model__min_samples_leaf": randint(1, 8),        # min samples per leaf
#     "model__max_features": ["sqrt", "log2"],         # feature sampling strategy
#     "model__bootstrap": [True, False]                # use bootstrapping or not
# }

# So far best parameter distribution based on previous runs to focus search space
rf_param_dist = {
    "model__n_estimators": [467],        
    "model__max_depth": [32],              
    "model__min_samples_split": [9],      
    "model__min_samples_leaf": [1],        
    "model__max_features": ["sqrt"],         
    "model__bootstrap": [False]                
}

cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Randomized search setup
rf_random = RandomizedSearchCV(
    estimator=rf_pipe_fe,
    param_distributions=rf_param_dist,
    n_iter=1,                         # reduced number of random combinations to 1 because of fixed set after previous runs
    scoring="neg_mean_absolute_error", 
    cv=cv,
    n_jobs=1,
    random_state=42,
    verbose=2
)

# Fit the search
rf_random.fit(X_train, y_train)

# Best model
rf_best_rand = rf_random.best_estimator_
print("Best Params:", rf_random.best_params_)

# Evaluate on validation set
rf_val_pred = rf_best_rand.predict(X_val)
print_metrics(y_val, rf_val_pred)

# Save best model
# joblib.dump(rf_best_rand, "rf_best.pkl")

##### 5.6 StackingRegressor

In [0]:
# Hyperparameter Tuning: StackingRegressor

# Old parameter distribution
# stack_param_dist = {
#     "final_estimator__learning_rate": uniform(0.02, 0.08),
#     "final_estimator__max_depth": randint(3, 7),
#     "final_estimator__min_samples_leaf": randint(3, 15),
#     "final_estimator__l2_regularization": uniform(0.0, 1.0)
# }

# So far best parameter distribution based on previous runs to focus search space
stack_param_dist = {
    "final_estimator__learning_rate": [0.0960571445127933],
    "final_estimator__max_depth": [5],
    "final_estimator__min_samples_leaf": [10],
    "final_estimator__l2_regularization": [0.3745401188473625]
}

stack_random = RandomizedSearchCV(
    estimator=stack_pipe_fe,
    param_distributions=stack_param_dist,
    n_iter=25,
    scoring="neg_mean_absolute_error",
    cv=3,                 # low CV because stacking is slow
    n_jobs=-1,
    random_state=42,
    verbose=3
)

stack_random.fit(X_train, y_train)

stack_best = stack_random.best_estimator_
print("StackingRegressor Best Params:", stack_random.best_params_)

stack_val_pred = stack_best.predict(X_val)
print_metrics(y_val, stack_val_pred)

# joblib.dump(stack_best, "stack_best.pkl")

### 6. Feature Importance of Tree Models (with SHAP)

  **Problem:** Current feature selection targets linear models
  (ElasticNet), but we primarily use tree-based models (HGB,
  RandomForest).

  **Solution:** Use SHAP (SHapley Additive exPlanations) to
  identify feature importance specifically for tree models

  **Why SHAP for Trees:**
  - Provides exact feature importance values for tree-based
  models
  - Tree models handle irrelevant features, but noise features
  still impact performance
  - Enables data-driven selection rather than statistical filter
  methods

In [0]:
def plot_shap_bar(top_df, title, cmap):
    """
    Horizontal bar plot for SHAP feature importances.

    Parameters
    ----------
    top_df : pd.DataFrame
        DataFrame with columns ["feature", "importance"], already sorted and sliced.
    title : str
        Plot title.
    cmap : matplotlib colormap
        Colormap used for bar coloring (e.g. plt.cm.Blues, plt.cm.Greens).
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.barh(
        top_df["feature"],
        top_df["importance"],
        color=cmap(np.linspace(0.4, 0.9, len(top_df)))
    )
    ax.set_xlabel("Average |SHAP| value")
    ax.set_ylabel("Feature")
    ax.set_title(title)
    ax.bar_label(bars, fmt="%.0f", padding=3)
    plt.tight_layout()
    plt.show()


#### 6.1 HGB

##### Step 1: Baseline Performance with Optimized Hyperparameters

In [0]:
X_val_processed = hgb_best.named_steps["preprocess"].transform(X_val)
hgb_val_pred = hgb_best.named_steps["model"].predict(X_val_processed)
n_features_total = X_val_processed.shape[1]
baseline_mae = mean_absolute_error(y_val, hgb_val_pred)

print("Baseline Performance of HGB model after Hyperparameter Tuning:\n")
print_metrics(y_val, hgb_val_pred)
print(f"\nTotal features used: {n_features_total}")

##### Step 2: SHAP Feature Importance Analysis

We use SHAP's TreeExplainer to calculate feature importance values. TreeExplainer is specifically optimized for tree-based models and provides exact Shapley values efficiently.

In [0]:
# Set seeds for reproducibility, otherwise we get different results each time we run the notebook
np.random.seed(42)

# Extract trained model and preprocessed data
hgb_model = hgb_best.named_steps["model"]
X_train_processed = hgb_best.named_steps["preprocess"].transform(X_train)

# Build feature names manually (avoid GroupMedianImputer issue)
feature_names_all = (
    log_features
    + numeric_features
    + list(
        hgb_best.named_steps["preprocess"]
        .named_transformers_["cat"]
        .named_steps["encoder"]
        .get_feature_names_out(categorical_features)
    )
)

# TreeExplainer for HGB model
explainer = shap.TreeExplainer(hgb_model)

# Sample for speed
sample_size = min(1000, len(X_train_processed))
sample_indices = np.random.choice(len(X_train_processed), sample_size, replace=False)
X_sample = X_train_processed[sample_indices]

print(f"Computing SHAP values for HGB on {sample_size} samples...")
shap_values = explainer.shap_values(X_sample)

# Mean absolute SHAP = global importance
feature_importance = np.abs(shap_values).mean(axis=0)

# Create DataFrame with proper columns
shap_importance_df = (
    pd.DataFrame({
        "feature": feature_names_all,
        "importance": feature_importance
    })
    .sort_values("importance", ascending=False)
    .reset_index(drop=True)
)

print("\nTop 20 most important features:")
print(shap_importance_df.head(20).to_string(index=False))

##### Step 3: Automated Feature Selection Optimization

Now we systematically test different numbers of top features to find the optimal subset. We train the model with the same optimized hyperparameters but using only the most important features identified by SHAP.

In [0]:
print("\nHGB Feature Selection Analysis with different bins:")

results = []
total_features = len(feature_names_all)

# Test range — adjust as you like
# feature_counts = list(range(5, total_features + 1))
feature_counts = list(range(16,20))

# Define bins to print # TODO bins werden nicht geprinted
print_bins = {5, 10, 15, 20, 30, 50, 70, 100, total_features}

# Use the same processed validation data
X_val_processed = hgb_best.named_steps["preprocess"].transform(X_val)


hgb_model = hgb_best.named_steps["model"]
clean_params = hgb_model.get_params()

# Track best model
best_model = None
best_mae = float("inf")
best_n = None
best_features = None

for n_features in feature_counts:
    # Select top N features
    top_features = shap_importance_df.head(n_features)["feature"].tolist()
    feature_indices = [i for i, fname in enumerate(feature_names_all) if fname in top_features]

    X_train_subset = X_train_processed[:, feature_indices]
    X_val_subset   = X_val_processed[:, feature_indices]

    # Train new model on subset of features
    hgb_selected = HistGradientBoostingRegressor(**clean_params)
    hgb_selected.fit(X_train_subset, y_train)

    pred_subset = hgb_selected.predict(X_val_subset)
    mae_subset = mean_absolute_error(y_val, pred_subset)

    # Save result
    results.append({"n_features": n_features, "mae": mae_subset})

    # Check if it's the best so far
    if mae_subset < best_mae:
        best_mae = mae_subset
        best_n = n_features
        best_model = hgb_selected
        best_features = top_features

    # Print only key bins
    if n_features in print_bins:
        improvement = baseline_mae - mae_subset
        print(f"Top {n_features:3d} features: MAE: {mae_subset:.1f} (Δ: {improvement:+.1f})")

# Convert results to DataFrame
results_df = pd.DataFrame(results)

print(f"\nOptimal feature selection results:")
print(f"Best performance with {best_n} features: MAE: {best_mae:.2f}")
print(f"Improvement over baseline: {baseline_mae - best_mae:+.2f} MAE\n")

print(f"Optimal {best_n} features for production model:")
for i, feat in enumerate(best_features, start=1):
    imp = shap_importance_df.loc[shap_importance_df['feature'] == feat, 'importance'].values[0]
    print(f"{i:2d}. {feat:25s} ({imp:.1f})")

In [0]:
top_k = 20
top_df = shap_importance_df.head(top_k).iloc[::-1]
plot_shap_bar(
    top_df,
    title=f"Top {top_k} HGB features by SHAP importance",
    cmap=plt.cm.Blues
)

In [0]:
# Build the final pipeline with feature selection included
def select_best_features(X):
    idx = [i for i, fname in enumerate(feature_names_all) if fname in best_features]
    return X[:, idx]

hgb_final_pipe = Pipeline([
    ("preprocess", hgb_best.named_steps["preprocess"]),
    ("feature_selector", FunctionTransformer(select_best_features, validate=False)),
    ("model", best_model)
])

# Save the best model for later use
joblib.dump(hgb_final_pipe, "hgb_best_feature.pkl")

#### 6.2 RF

##### Step 1: Baseline Performance with Optimized Hyperparameters

In [0]:
# Use the tuned RF pipeline (rf_best_rand) and compute baseline on the validation set
X_val_processed_rf = rf_best_rand.named_steps["preprocess"].transform(X_val)
rf_val_pred = rf_best_rand.named_steps["model"].predict(X_val_processed_rf)
n_features_total_rf = X_val_processed_rf.shape[1]
baseline_mae_rf = mean_absolute_error(y_val, rf_val_pred)

print("Baseline Performance of RF model after Hyperparameter Tuning:\n")
print_metrics(y_val, rf_val_pred)
print(f"\nTotal features used: {n_features_total_rf}")

##### Step 2: SHAP Feature Importance Analysis

In [0]:
# Set seeds for reproducibility, otherwise we get different results each time we run the notebook
np.random.seed(42)

# Preprocess training data (same preprocessor as in rf_best_rand)
X_train_processed_rf = rf_best_rand.named_steps["preprocess"].transform(X_train)

# Build feature names (log + numeric + OHE categories), same logic as for HGB
feature_names_all_rf = (
    log_features
    + numeric_features
    + list(
        rf_best_rand.named_steps["preprocess"]
        .named_transformers_["cat"]
        .named_steps["encoder"]
        .get_feature_names_out(categorical_features)
    )
)

# SHAP for RandomForest
rf_model = rf_best_rand.named_steps["model"]
explainer_rf = shap.TreeExplainer(rf_model)

sample_size_rf = min(50, len(X_train_processed_rf))
sample_idx_rf = np.random.choice(len(X_train_processed_rf), sample_size_rf, replace=False)
X_sample_rf = X_train_processed_rf[sample_idx_rf]

print(f"Computing SHAP values for RF on {sample_size_rf} samples...")
shap_values_rf = explainer_rf.shap_values(X_sample_rf)

# Mean absolute SHAP = global importance
feature_importance_rf = np.abs(shap_values_rf).mean(axis=0)

# Create DataFrame with proper columns
shap_importance_df_rf = (
    pd.DataFrame({
        "feature": feature_names_all_rf,
        "importance": feature_importance_rf
    })
    .sort_values("importance", ascending=False)
    .reset_index(drop=True)
)

print("\nTop 20 most important RF features:")
print(shap_importance_df_rf.head(20).to_string(index=False))

##### Step 3: Automated Feature Selection Optimization

In [0]:
# TODO Step 2 (6 min) and Step 3 (9 min) vs HGB Step 2+3 40 seconds => code needs to be refined, cant be that longer

In [0]:
print("\nRF Feature Selection Analysis with different bins:")

results_rf = []

# Use the same processed validation data
X_val_processed_rf = rf_best_rand.named_steps["preprocess"].transform(X_val)

# Reuse tuned RF hyperparameters
rf_params = {k.replace("model__", ""): v for k, v in rf_random.best_params_.items()}

# Track best model
best_model_rf = None
best_mae_rf = float("inf")
best_n_rf = None
best_features_rf = None

# feature_counts_rf = list(range(5, total_features + 1)) 
feature_counts_rf = list(range(16, 30)) #faster
print_bins_rf = {5, 10, 15, 20, 30, 50, 70, 100, len(feature_names_all_rf)}

for n_features in feature_counts_rf:
    # Select top N features
    top_features_rf = shap_importance_df_rf.head(n_features)["feature"].tolist()
    feature_indices_rf = [i for i, fname in enumerate(feature_names_all_rf) if fname in top_features_rf]

    X_train_subset_rf = X_train_processed_rf[:, feature_indices_rf]
    X_val_subset_rf   = X_val_processed_rf[:, feature_indices_rf]

    # Train RF with the same tuned params
    rf_selected = RandomForestRegressor(random_state=42, n_jobs=-1, **rf_params)
    rf_selected.fit(X_train_subset_rf, y_train)
    pred_subset_rf = rf_selected.predict(X_val_subset_rf)
    mae_subset_rf = mean_absolute_error(y_val, pred_subset_rf)

    # Save result
    results_rf.append({"n_features": n_features, "mae": mae_subset_rf})

    # Best so far?
    if mae_subset_rf < best_mae_rf:
        best_mae_rf = mae_subset_rf
        best_n_rf = n_features
        best_model_rf = rf_selected
        best_features_rf = top_features_rf

    # Optional summary for key bins
    if n_features in print_bins_rf:
        improvement_rf = baseline_mae_rf - mae_subset_rf
        print(f"Top {n_features:3d} features: MAE: {mae_subset_rf:.1f} (Δ: {improvement_rf:+.1f})")

results_rf_df = pd.DataFrame(results_rf)

print(f"\nOptimal RF feature selection results:")
print(f"Best performance with {best_n_rf} features: MAE: {best_mae_rf:.2f}")
print(f"Improvement over baseline: {baseline_mae_rf - best_mae_rf:+.2f} MAE\n")

print(f"Optimal {best_n_rf} RF features for production model:")
for i, feat in enumerate(best_features_rf, start=1):
    imp = shap_importance_df_rf.loc[shap_importance_df_rf['feature'] == feat, 'importance'].values[0]
    print(f"{i:2d}. {feat:25s} ({imp:.1f})")

In [0]:
# RF SHAP bar plot
top_k = 20
top_df = shap_importance_df_rf.head(top_k).iloc[::-1]
plot_shap_bar(
    top_df,
    title=f"Top {top_k} RF features by SHAP importance",
    cmap=plt.cm.Greens
)

In [0]:
# Save the best RF model for later use

# Build the final RF pipeline with feature selection included
def select_best_features_rf(X):
    idx = [i for i, fname in enumerate(feature_names_all_rf) if fname in best_features_rf]
    return X[:, idx]

final_rf_pipe = Pipeline([
    ("preprocess", rf_best_rand.named_steps["preprocess"]),
    ("feature_selector", FunctionTransformer(select_best_features_rf, validate=False)),
    ("model", best_model_rf)
])

# joblib.dump(final_rf_pipe, "rf_best_feature.pkl")

In [0]:
# TODO check whether this might improve performance with big n_iter when everything else is done. If not, just remove section 7.
hgb_random = hgb_hyperparameter_tuning(hgb_final_pipe, n_iter=1000)
# Get best model
hgb_best = hgb_random.best_estimator_
print("Best Params:", hgb_random.best_params_)

# Evaluate on validation set
hgb_val_pred = hgb_random.best_estimator_.predict(X_val)
print_metrics(y_val, hgb_val_pred)

**Build Final Stacking Regressor to mix tuned and feature selected HGB and RF**

In [0]:
stack_pipe_final = StackingRegressor(
    estimators=[
        ("hgb_final", hgb_final_pipe),   # tuned HGB pipeline (preprocessor + model)
        ("rf_final",  final_rf_pipe),    # tuned RF pipeline (preprocessor + model)
    ],
    final_estimator=LinearRegression(),  # simple, perfect for 2 base preds
    passthrough=False,                   # meta model sees only base predictions
    cv=5,                                # proper OOF stacking
    n_jobs=1                             # no BrokenProcessPool on Databricks
)

stack_pipe_final.fit(X_train, y_train)
stack_val_pred = stack_pipe_final.predict(X_val)
print_metrics(y_val, stack_val_pred)

# MAE: 1255.3112 | RMSE: 4157099.9081 | R2: 0.9544

# Kaggle Score submit 1274 !! OVERFITTED

Final SR of the tuned HGB and RF models, did not improve over the best single HGB and RF models on the validation set (MAE 1332 vs 1281/1284). 

Therefore we will keep the RF/HGB model => with such small difference in MAE, we further need to evaluate them both

### 7. Kaggle Competition

In [0]:
# Load best Models from Joblib

hgb_best_99 = joblib.load("hgb_best_feature.pkl")

In [0]:
# Pick best model and predict on test:

# Build a temporary train df with the target for anchor computation
train_tmp = X_train.copy()
train_tmp["price"] = y_train

# Compute anchors based on training data and map them onto the test set
_, df_cars_test_anchors = add_price_anchor_features(
    train_tmp,
    df_cars_test.copy(),
    target_col="price"
)

# Now we can safely predict on the test set with all expected columns present
df_cars_test_anchors["price"] = hgb_best_99.predict(df_cars_test_anchors)

# Save submission file
df_cars_test_anchors["price"].to_csv("Group05_Version05.csv", index=True)

In [0]:
!kaggle competitions submit -c cars4you -f Group05_Version05.csv -m "Message"

In [0]:
!kaggle competitions submissions -c cars4you

### 8. Load and predict model

In [0]:
hgb_loaded = joblib.load("hgb_best_feature.pkl")

preprocessor = hgb_loaded.named_steps["preprocess"]
display(preprocessor)

pred_loaded = hgb_loaded.predict(X_val)
mae_loaded = mean_absolute_error(y_val, pred_loaded)
print(f"Loaded model MAE on validation set: {mae_loaded:.2f}")